<div style="font-size:18pt; padding-top:20px; text-align:center">СЕМИНАР. <b>Кросс-валидация</b></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@mail.ru)</span></div>

### Содержание

- [1. Подходы к разбиению исходных данных](#1.-Подходы-к-разбиению-исходных-данных)
- [2. Оценка качества модели](#2.-Оценка-качества-модели)
- [3. Выбор модели](#3.-Выбор-модели)
- [4. Сетка гиперпараметров](#4.-Сетка-гиперпараметров)
- [5. Вложенная кросс-валидация](#5.-Вложенная-кросс-валидация)
- [6. Источники](#6.-Источники)

<p><b>Подключение библиотек</b></p>

In [ ]:
from sklearn import datasets
from scipy import stats
import numpy as np

from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from sklearn.model_selection import (
    train_test_split,
    ShuffleSplit,
    KFold, 
    LeaveOneOut,
    StratifiedKFold,
    cross_val_score, 
    cross_validate
)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

## 1. Подходы к разбиению исходных данных

### Отложенная выборка (Holdout)

In [ ]:
# Исходные данные
x = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
y = np.array([1, 1, 0, 0, 1, 0, 1, 1, 0, 0])

In [ ]:
# Разделения данные на обучающее и тестовое множества
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
print(x_train, y_train)
print(x_test, y_test)

In [ ]:
# Замечание: данный класс может быть использован в GridSearchCV 
# для реализации выбора параметров с отложенной выборкой
splitter = ShuffleSplit(n_splits=1, test_size=0.3)
splits = splitter.split(x)
for train_index, test_index in splits:
    print(train_index, test_index)
    # print(x[train_index], x[test_index])

### Кросс-валидация K-Folds

In [ ]:
# Исходные данные
x = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
y = np.array([1, 1, 0, 0, 1, 0, 1, 1, 0, 0])

In [ ]:
# Без тасовки
kf = KFold(n_splits=3)
splits = kf.split(x)
for train_index, test_index in splits:
    print(train_index, test_index)

In [ ]:
# С тасовки
kf = KFold(n_splits=3, shuffle=True, random_state=0)
splits = kf.split(x)
for train_index, test_index in splits:
    print(train_index, test_index)

In [ ]:
# Вывод сплитов
splits = kf.split(x, y)
for indx, (train_index, test_index) in enumerate(splits):
    print("Split", indx+1)
    print("\tTrain set")
    print("\tx: {}\n\ty: {}".format(x[train_index],  y[train_index]))
    print("\tTest set")
    print("\tx: {}\n\ty: {}\n".format(x[test_index],  y[test_index]))

In [ ]:
# Тасовка и кросс-валидация
from sklearn.utils import shuffle

x_, y_ = shuffle(x, y)
print("Initial x:\t{}\nShuffled x:\t{}".format(x, x_))
print("Initial y:\t{}\nShuffled y:\t{}".format(y, y_))

kf = KFold(n_splits=3)
splits = kf.split(x_)

### Кросс-валидация Leave-One-Out (LOO)

In [ ]:
# Исходные данные
x = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
y = np.array([1, 1, 0, 0, 1, 0, 1, 1, 0, 0])

In [ ]:
loo = LeaveOneOut()
for train, test in loo.split(x):
    print("{}{}".format(train, test))

### Стратифицированная выборка

In [ ]:
x = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
y = np.array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
# Кросс-валидация с kfolds без тасовки
kf = KFold(n_splits=3)
splits = kf.split(x, y)
i = 0
for train_index, test_index in splits:
    print("Split", i+1)
    print("\tindices:\t{}{}".format(train_index, test_index))
    print("\ty:\t\t{}{}".format(y[train_index], y[test_index]))
    i += 1

In [ ]:
# Стратифицированная кросс-валидация с kfolds
kf = StratifiedKFold(n_splits=3)
splits = kf.split(x, y)
i = 0
for train_index, test_index in splits:
    print("Split", i+1)
    print("\tindices:\t{}{}".format(train_index, test_index))
    print("\ty:\t\t{}{}".format(y[train_index], y[test_index]))
    i += 1

## 2. Оценка качества модели

In [ ]:
def regression_dataset(intercept=2, slope=0.3, n=100, start_x=4, length_x=8, mu=0, sigma=0.5):
    f = lambda x: intercept + slope*x
    x = stats.uniform.rvs(size=n, loc=start_x, scale=length_x)
    e = stats.norm.rvs(size=n, loc=mu, scale=sigma)
    y_true = f(x) + e
    return x.reshape(-1,1), y_true, f

# Генерация исходных данных
X, y_true, f = regression_dataset()

In [ ]:
# График
xx = np.linspace(4, 12, 100)
plt.title("Initial Data")
plt.scatter(X, y_true, color="green", label="observed")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function", zorder=1)
plt.grid(True)
plt.xlabel("$x$")
plt.ylabel("$y_{true}$")
plt.legend()
plt.show()

### Holdout

In [ ]:
REPEATS = 10

mses = np.full(REPEATS, np.inf)

for i in range(REPEATS):

    # Разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X, y_true, 
        test_size=0.3, 
        random_state=i)

    # Обучение
    linear_model = LinearRegression()
    linear_model.fit(X_train, y_train)
    
    # Предсказания для тестового множества
    y_test__pred = linear_model.predict(X_test)

    # Ошибки на тестовом множестве
    mses[i] = mean_squared_error(y_test, y_test__pred)
    print("{}) Test MSE: {}".format(i+1, mses[i]))

print("\nДоверительный интервал MSE: {:0.3f} +/- {:0.3f}"
      .format(mses.mean(), mses.std() * 2.0))

# Построение диаграммы размаха
holdout_mses = mses
plt.figure()
plt.scatter(np.full(len(holdout_mses), 1), holdout_mses, c="green", alpha=0.5)
plt.boxplot(holdout_mses, showmeans=True, labels=["holdout", ],
            meanprops=dict(markerfacecolor="orange", markeredgecolor="black"))
plt.ylabel("MSE")
plt.grid(True)
plt.show()

### K-Folds

In [ ]:
REPEATS = 10

mses = np.full(REPEATS, np.inf)
kf = KFold(n_splits=5, shuffle=True)

for i in range(REPEATS):
    
    kf.random_state = i
    
    # Обучение
    mse_list = - cross_val_score(linear_model, X, y_true, cv=kf, 
                                 scoring="neg_mean_squared_error")
    
     # Средняя ошибка на тестовом множестве
    mses[i] = mse_list.mean()
    print("{}) Test MSE: {}".format(i+1,  mses[i]))
    
print("\nДоверительный интервал MSE: {:0.3f} +/- {:0.3f}".format(mses.mean(), mses.std() * 2.0))

# Построение диаграммы размаха
plt.figure()
plt.scatter(np.full(len(mses), 1), mses, c="steelblue", alpha=0.5)
plt.boxplot(mses, showmeans=True, labels=["kfolds cv", ],
            meanprops=dict(markerfacecolor="orange", markeredgecolor="black"))
plt.ylabel("MSE")
plt.grid(True)
plt.show()

In [ ]:
# Сравнение оценки качества модели с отложенной выборкой и кросс-валидацией с kfolds
plt.figure()
plt.title("holdout vs kfolds cv")
plt.scatter(np.full(len(holdout_mses), 1), holdout_mses, c="green", alpha=0.5)
plt.scatter(np.full(len(mses), 2), mses, c="steelblue", alpha=0.5)
plt.boxplot(np.c_[holdout_mses, mses], showmeans=True,
            labels=["holdouts", "kfolds cv"],
            meanprops=dict(markerfacecolor="orange", markeredgecolor="black"))
plt.ylabel("MSE")
plt.grid(True)
plt.show()

## 3. Выбор модели

In [ ]:
def regression_dataset():
    n = 200
    x = stats.uniform.rvs(size=n, loc=0, scale=5, random_state=10)
    f = lambda x:  np.sin(x)
    y_true = stats.norm.rvs(size=n, loc=0, scale=0.4, random_state=10) + f(x)
    return x.reshape(-1,1), y_true, f

# Генерация исходных данных
X, y_true, f = regression_dataset()

# График
xx = np.linspace(0, 5, 100)
plt.title("Initial Data")
plt.scatter(X, y_true, color="green", label="observed")
plt.plot(xx, f(xx), "-", color="SteelBlue", label="true function", zorder=1)
plt.grid(True)
plt.xlabel("$x$")
plt.ylabel("$y_{true}$")
plt.legend()
plt.show()

In [ ]:
RANDOM_STATE = 12345
SPLITS = 5
MAX_DEGREE = 9

#### Вариант 1

Замечание: Важно, чтобы сплиты `KFold` были одинаковыми для всех степеней. Поэтому далее используется `random_state`. Либо можно перетасовать данные перед кросс-валидацией и установить `shuffle` равным `False` для `KFold`

In [ ]:
# Разбиение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(
    X, y_true, test_size=0.3, random_state=RANDOM_STATE)

# Инициализация делителя для кросс-валидации
# Замечание: нет необходимости использовать shuffle, т.к. train_test_split
# выполняет тасовку
kf = KFold(n_splits=SPLITS, shuffle=True, random_state=RANDOM_STATE)

# Степени полинома
degrees = list(range(1, MAX_DEGREE+1))

best_degree = 0
best_mse = float("inf")

# Замечание: будем использовать один экземпляр и менять степень;
#  каждый раз модель будет заново обучаться; можно и каждый раз
#  создавать отдельный экземпляр
pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=None, include_bias=False)), 
    ("linear_model", LinearRegression(fit_intercept=True))
])

# Выбор степени с наименьшей валидационной ошибкой 
for indx, degree in enumerate(degrees):
    # Обучение с кросс-валидацией
    pipeline.named_steps["transformation"].degree = degree
    scores = cross_validate(
        pipeline, X_train, y_train, cv=kf, 
        scoring=["neg_mean_squared_error",],
        return_train_score=True
    )
    # Средняя ошибка на проверочном множестве
    mse_avg = -scores["test_neg_mean_squared_error"].mean()
    # Выбор лучшего гиперпараметра
    if best_mse > mse_avg:
        best_mse = mse_avg
        best_degree = degree
    print("{}) Val MSE for degree {}: {}".format(indx+1, degree, mse_avg))

print("Best degree:", best_degree)

# Повторное обучение на всем обучающем множестве 
pipeline.named_steps["transformation"].degree = best_degree
pipeline.fit(X_train, y_train)

# Предсказания для тестового множества
y_test__pred = pipeline.predict(X_test)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)

print("Тестовое множество:")
print("\tTest MSE:", mse_test)

#### Вариант 2

In [ ]:
# Разбиение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(
    X, y_true, test_size=0.3, random_state=RANDOM_STATE)

# Инициализация делителя для кросс-валидации
kf = KFold(n_splits=SPLITS, shuffle=True, random_state=RANDOM_STATE)

# Степени полинома
degrees = list(range(1, MAX_DEGREE+1))

# Инициализация массива для MSE
scores = np.zeros((MAX_DEGREE, SPLITS))

pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=None, include_bias=False)), 
    ("linear_model", LinearRegression(fit_intercept=True))
])

# Обучение
for i, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    for j, degree in enumerate(degrees):
        pipeline.named_steps["transformation"].degree = degree
        pipeline.fit(X_train[train_index], y_train[train_index])
        scores[j, i] = mean_squared_error(
            y_train[val_index], 
            pipeline.predict(X_train[val_index]))

# Средние проверочные ошибки для каждой степени
mses_avg = scores.mean(axis=1)

for indx, mse_avg in enumerate(mses_avg):
    print("{}) Val MSE for degree {}: {}".format(indx+1, degrees[indx], mse_avg))

indx_min__mse_avg = mses_avg.argmin()
best_degree = degrees[indx_min__mse_avg]

print("Best degree:", best_degree)

# Повторное обучение на всем обучающем множестве 
pipeline.named_steps["transformation"].degree = best_degree
pipeline.fit(X_train, y_train)

# Предсказания для тестового множества
y_test__pred = pipeline.predict(X_test)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)

print("Тестовое множество:")
print("\tTest MSE:", mse_test)

#### Вариант 3

In [ ]:
from joblib import Parallel, delayed

In [ ]:
def scores_on_split(train_index, val_index, max_degree=MAX_DEGREE):
    pipeline = Pipeline([
        ("transformation", PolynomialFeatures(degree=None, include_bias=False)), 
        ("linear_model", LinearRegression(fit_intercept=True))
    ])
    degrees = list(range(1, MAX_DEGREE+1))
    scores = np.zeros(max_degree)
    for j, degree in enumerate(degrees):
        pipeline.named_steps["transformation"].degree = degree
        pipeline.fit(X_train[train_index], y_train[train_index])
        scores[j] = mean_squared_error(
            y_train[val_index], 
            pipeline.predict(X_train[val_index]))
    return scores

In [ ]:
# Разбиение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(
    X, y_true, test_size=0.3, random_state=RANDOM_STATE)

# Инициализация делителя для кросс-валидации
kf = KFold(n_splits=SPLITS, shuffle=True, random_state=RANDOM_STATE)

# Инициализация массива для MSE
scores = list()

# Параллельное обучение по сплитам
parallel = Parallel(n_jobs=4)
func = delayed(scores_on_split)
scores = np.asarray(
    parallel(
        func(train_index, val_index) 
        for train_index, val_index in kf.split(X_train, y_train)
    )
)

# Средние проверочные ошибки для каждой степени
mses_avg = scores.mean(axis=0)

for indx, mse_avg in enumerate(mses_avg):
    print("{}) Val MSE for degree {}: {}".format(indx+1, degrees[indx], mse_avg))

indx_min__mse_avg = mses_avg.argmin()
best_degree = degrees[indx_min__mse_avg]

print("Best degree:", best_degree)

# Повторное обучение на всем обучающем множестве 
pipeline.named_steps["transformation"].degree = best_degree
pipeline.fit(X_train, y_train)

# Предсказания для тестового множества
y_test__pred = pipeline.predict(X_test)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)

print("Тестовое множество:")
print("\tTest MSE:", mse_test)

## 4. Сетка гиперпараметров

`GridSearchCV`

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Разбиение данных на обучающее и тестовое множества
X_train, X_test, y_train, y_test = train_test_split(
    X, y_true, test_size=0.3, random_state=RANDOM_STATE)

# Инициализация делителя для кросс-валидации
kf = KFold(n_splits=SPLITS, shuffle=True, random_state=RANDOM_STATE)

# Степени полинома
degrees = list(range(1, MAX_DEGREE+1))

# Инициализация массива для MSE
scores = np.zeros((MAX_DEGREE, SPLITS))

pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=None, include_bias=False)), 
    ("linear_model", LinearRegression(fit_intercept=True))
])

# Сетка параметров
parameters = {
    "transformation__degree": degrees,
}

# Параметры обучения
grid_class_parameters = {
    "estimator": pipeline,
    "param_grid": parameters,
    "cv": kf,
    "scoring": "neg_mean_squared_error"
}

# Обучение
grid_search = GridSearchCV(**grid_class_parameters)
grid_search.fit(X_train, y_train)

# Средние проверочные ошибки для каждой степени
mses_avg = np.abs(grid_search.cv_results_["mean_test_score"])

for indx, mse_avg in enumerate(mses_avg):
    print("{}) Val MSE for degree {}: {}".format(indx+1, degrees[indx], mse_avg))

print("Best degree:", grid_search.best_params_["transformation__degree"])

# Предсказания для тестового множества
y_test__pred = grid_search.predict(X_test)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)

print("Тестовое множество:")
print("\tTest MSE:", mse_test)

`RandomizedSearchCV`

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [ ]:
# Закон распределения 
# Дискретное равномерное распределение (выборка с возвращением)
degree_rv = randint(low=1, high=20)

In [ ]:
distributions = {
    'transformation__degree': degree_rv,
}
rand_search = RandomizedSearchCV(
    estimator=pipeline, 
    param_distributions=distributions,
    n_iter=5,
    cv=kf,
    scoring='neg_mean_squared_error',
    random_state=RANDOM_STATE
)
rand_search.fit(X_train, y_train)

In [ ]:
# Средние проверочные ошибки для каждой степени
mses_avg = np.abs(grid_search.cv_results_["mean_test_score"])

for indx, mse_avg in enumerate(mses_avg):
    print("{}) Val MSE for degree {}: {}".format(indx+1, degrees[indx], mse_avg))

print("Best degree:", rand_search.best_params_["transformation__degree"])

In [ ]:
rand_search.cv_results_['params']

In [ ]:
rand_search.best_params_

In [ ]:
-rand_search.best_score_

In [ ]:
# Предсказания для тестового множества
y_test__pred = rand_search.predict(X_test)

# Ошибки на тестовом множестве
mse_test = mean_squared_error(y_test, y_test__pred)

print("Тестовое множество:")
print("\tTest MSE:", mse_test)

## 5. Вложенная кросс-валидация

### Оценка качества модели

In [ ]:
# Выполним тасовку данных
X, y_true = shuffle(X, y_true, random_state=RANDOM_STATE)

#### Вариант 1

In [ ]:
# Количество сплитов для внешней кросс-валидации
OUTER_SPLITS = 5
# Количество сплитов для внутренней кросс-валидации
INNER_STPLITS = 3

# Степени полинома (гиперпараметр)
degrees = list(range(1, MAX_DEGREE+1))

# Инициализация массива для MSE
outer_scores = np.zeros(OUTER_SPLITS)
inner_scores = np.zeros((MAX_DEGREE, INNER_STPLITS))

# Модель с гиперпараметрами
pipeline = Pipeline([
    ("transformation", PolynomialFeatures(degree=None, include_bias=False)), 
    ("linear_model", LinearRegression(fit_intercept=True))
])

# Делитель данных для внешней и внутренней кросс-валидации
outer_kf = KFold(n_splits=OUTER_SPLITS, shuffle=True, random_state=RANDOM_STATE)
inner_kf = KFold(n_splits=INNER_STPLITS, shuffle=True, random_state=RANDOM_STATE)

# Внешняя кросс-валидация
for i, (trainval_index, test_index) in enumerate(outer_kf.split(X, y_true)):
    # Внутренняя кросс-валидация
    for j, (train_index, val_index) in enumerate(inner_kf.split(X[trainval_index], y_true[trainval_index])):
        for k, degree in enumerate(degrees):
            pipeline.named_steps["transformation"].degree = degree
            pipeline.fit(X[trainval_index][train_index], y_true[trainval_index][train_index])
            inner_scores[k, j] = mean_squared_error(
                y_true=y_true[trainval_index][val_index], 
                y_pred=pipeline.predict(X[trainval_index][val_index])
            )
    # Выбор гиперпараметра
    best_inner_degree = degrees[inner_scores.mean(axis=1).argmin()]
    # Повторное обучение на всем trainval
    pipeline.named_steps["transformation"].degree = best_inner_degree
    pipeline.fit(X[trainval_index], y_true[trainval_index])
    # Тестовая ошибка
    outer_scores[i] = mean_squared_error(
        y_true=y_true[test_index], 
        y_pred=pipeline.predict(X[test_index])
    )
    print(f'Test MSE for outer split {i}: {outer_scores[i]:.4f}\tBest degree: {best_inner_degree}')

# Ошибки на тестовом множестве (среднее по сплитам)
mse_test = outer_scores.mean()

print("Оценка ошибки тестирования:")
print("\tTest MSE:", mse_test)

#### Вариант 2

In [ ]:
# Сетка гиперпараметров
param_grid = { "transformation__degree": degrees, }

# Обучение
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=inner_kf,
    scoring="neg_mean_squared_error"
)

# Вложенная кросс-валидация
scores = cross_validate(
    grid_search, X, y_true, cv=outer_kf, 
    scoring=["neg_mean_squared_error",]
)

mse_test = -scores['test_neg_mean_squared_error'].mean()

print("Оценка ошибки тестирования:")
print("\tTest MSE:", mse_test)

### Выбор модели

#### Вариант 1

In [ ]:
# Инициализация массива для MSE
scores = np.zeros((MAX_DEGREE, OUTER_SPLITS))
for i, (trainval_index, test_index) in enumerate(outer_kf.split(X, y_true)):
    for k, degree in enumerate(degrees):
        pipeline.named_steps["transformation"].degree = degree
        pipeline.fit(X[trainval_index], y_true[trainval_index])
        scores[k, i] = mean_squared_error(
            y_true=y_true[test_index], 
            y_pred=pipeline.predict(X[test_index])
        )
best_mse = scores.mean(axis=1).min()
best_degree = degrees[scores.mean(axis=1).argmin()]
print(f'Best hyperparameter: {best_degree}\tMSE: {best_mse}')

In [ ]:
from sklearn.base import clone

# Обучение на всем наборе данных
best_pipeline = clone(pipeline)
best_pipeline.named_steps["transformation"].degree = best_degree
best_pipeline.fit(X[trainval_index], y_true[trainval_index])

# Модель для предсказания
best_pipeline

#### Вариант 2

In [ ]:
# Обучение
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=outer_kf,
    scoring="neg_mean_squared_error"
)

grid_search.fit(X, y_true)

In [ ]:
best_mse = -grid_search.best_score_
best_degree = grid_search.best_params_['transformation__degree']

print(f'Best hyperparameter: {best_degree}\tMSE: {best_mse}')

In [ ]:
# Модель для предсказания
grid_search

## 6. Источники

- [Sklearn: Cross-validation: evaluating estimator performance](http://scikit-learn.org/stable/modules/cross_validation.html)
- [Model selection done right: A gentle introduction to nested cross-validation](https://ploomber.io/blog/nested-cv/)